In [1]:

import os
os.environ["R_HOME"] = r"/Library/Frameworks/R.framework/Resources" 
import sys
sys.path.append("/Users/anishdhir/Downloads/Sloppy")
sys.path.append("/Users/anishdhir/Documents/gp-causal")
from data.get_data import get_tubingen_pairs_dataset
# change as needed#Import necessary packages
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
#Must be activated
pandas2ri.activate()
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import trange
from sklearn.metrics import roc_auc_score
from data.get_data import get_cha_pairs_dataset, get_gauss_pairs_dataset, get_multi_pairs_dataset
from data.get_data import get_net_pairs_dataset

In [2]:
from rpy2.robjects.packages import STAP
from data import get_data

In [3]:
WORKDIR = "/Users/anishdhir/Documents/gp-causal"

In [4]:
with open('/Users/anishdhir/Downloads/Sloppy/Sloppy.R', 'r') as f:
    string = f.read()
#Parse using STAP
sloppy = STAP(string, "Sloppy")

In [5]:
def return_pairs(data_name):
    synth_data_names = ["cha", "multi", "net", "gauss"]
    if data_name in synth_data_names:
        data_get = getattr(get_data, f"get_{data_name}_pairs_dataset")
        x, y, weight, target = data_get(data_path=f"{WORKDIR}/data/{data_name}_pairs/files")
    elif data_name == "tueb":
        data_get = getattr(get_data, f"get_tubingen_pairs_dataset")
        x, y, weight, target = data_get(data_path=f"{WORKDIR}/data/pairs/files")
    else:
        full_data_name = synth_data_names.append("tueb")
        raise ValueError(f"data_name variable must be in {full_data_name}")
    return x, y, weight, target
    

In [6]:
scores = {"AIC": sloppy.AICscore, "BIC": sloppy.BICscore}
# datafuncs = ["cha", "multi", "net", "gauss"]
datafuncs = ["tueb"]

In [7]:
def get_results(x, y, sloppy_score, lenx):
    epsilon = np.zeros(lenx)
    target = np.ones_like(epsilon)
    causd = []
    res = np.zeros(lenx)
    res_dict = {}
    counter = 0
    skipped = 0
    for i in range(lenx):
        if x[i].shape[-1] < 2:
            input = StandardScaler().fit_transform(x[i]).astype(np.float64)
            output = StandardScaler().fit_transform(y[i]).astype(np.float64) 
            data = np.concatenate((input, output), axis=1)
            score = sloppy_score
            answer = sloppy.SloppyS(t=data, scoreF=score)
            eps = answer.rx2("epsilon")
            causd.append(answer.rx2("cd"))
            result_x_y = answer.rx2("XtoY.meta")
            result_y_x = answer.rx2("YtoX.meta")
            if type(result_x_y) == np.ndarray:
                res[counter] = 0
                res_dict[i] = 0
            else:
                x_y_score = result_x_y.rx2("Score")
                y_x_score = result_y_x.rx2("Score")
                res[counter] = y_x_score - x_y_score
                res_dict[i] = y_x_score - x_y_score
            epsilon[counter] = eps
            counter += 1
        else:
            skipped += 1
            pass
    return res

In [8]:
def balance_for_auc(target, pred_scores):
    # Targets are {-1, 1}, need to make sure it sums to zero
    balance = int(np.sum(target))
    if balance != 0:
        # There are more negative examples
        if balance < 0:
            switch_cand_idx = np.nonzero(target < 0)[0]
        # There are more positive examples here
        else:
            switch_cand_idx = np.nonzero(target > 0)[0]
        # get "balance" number of indices
        switch_idx = np.random.choice(
            switch_cand_idx, size=int(np.abs(balance) // 2), replace=False
        )
        final_target = target.copy()
        final_target[switch_idx] *= -1
        final_pred_scores = pred_scores.copy()
        final_pred_scores[switch_idx] *= -1
    else:
        final_target = target.copy()
        final_pred_scores = pred_scores.copy()
    if (balance % 2) == 0:
        assert np.sum(final_target) == 0
    else:
        assert np.abs(np.sum(final_target)) == 1
    return final_target, final_pred_scores


def calculate_auc(target, pred_scores, num_shuffles=1000):
    # Need to make sure that the classes are evenly balanced
    auc_all = []
    for i in trange(num_shuffles):
        total_runs = len(target)
        flip_idx = np.random.choice(np.arange(total_runs), total_runs // 2, replace=False)
        for i in range(total_runs):
            if i in flip_idx:
                target[i] *= -1
                pred_scores[i] *= -1
        final_target, final_pred_scores = balance_for_auc(target, pred_scores)
        roc_auc = roc_auc_score(final_target, final_pred_scores)
        auc_all.append(roc_auc)
    return np.mean(auc_all)


In [13]:
resultlist = []
for now_score in ["AIC", "BIC"]:
    sl_score = scores[now_score]
    for dataname in datafuncs:
        x, y, weight, target = return_pairs(dataname)
        lenx = 99 if dataname == "tueb" else len(target)
        res = get_results(x, y, sl_score, lenx)
        target = np.ones(99) if dataname == "tueb" else target
        auc = calculate_auc(target, res, 1000)
        print(f"Score: {now_score}, Data: {dataname}, AUC: {auc}")
        resultlist.append((now_score, dataname, auc))

Load cause-effect pairs: 100%|██████████| 108/108 [00:00<00:00, 268.87it/s]


[1] 2.203881
[1] 2.110951
[1] 1
[1] 1
[1] 2.000534
[1] 2.024149
[1] 2.000343
[1] 2.000004
[1] 3.000397
[1] 3.000329
[1] 2.999663
[1] 1
[1] 2.006462
[1] 3.000373
[1] 2.017035
[1] 2.001231
[1] 1
[1] 2.094002
[1] 2.002141
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 2.077105
[1] 1
[1] 1
[1] 2.058063
[1] 2.058063
[1] 1
[1] 1
[1] 2.058063
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 2.000988
[1] 5.000367
[1] 2.068727
[1] 2.060701
[1] 2.031984
[1] 8.001126
[1] 2.008028
[1] 2.005998
[1] 2.005811
[1] 1
[1] 2.000111
[1] 2.000109
[1] 1
[1] 2.000094
[1] 2.000097
[1] 1
[1] 1
[1] 1
[1] 2.001951
[1] 1
[1] 2.486357
[1] 2.836536
[1] 1
[1] 2.000001
[1] 1
[1] 5.069562
[1] 2.999672
[1] 1
[1] 2.000527
[1] 2.104315
[1] 3.000358
[1] 2.999712
[1] 4.000425
[1] 2.016633
[1] 2.05809
[1] 2.178834
[1] 2.235389
[1] 1
[1] 2.002511
[1] 3.000362
[1] 2.004991
[1] 2.003684
[1] 1
[1] 2.014646
[1] 2.016681
[1] 2.118808
[1] 5.000585
[1] 5.998985
[1] 4.000549
[1] 1
[1] 2.0012
[1] 2.000011


100%|██████████| 1000/1000 [00:00<00:00, 1937.67it/s]


Score: AIC, Data: tueb, AUC: 0.6529342857142857


Load cause-effect pairs: 100%|██████████| 108/108 [00:00<00:00, 341.62it/s]

[1]

 1
[1] 1
[1] 1
[1] 1
[1] 2.000534
[1] 2.024149
[1] 2.000343
[1] 1
[1] 2.071699
[1] 2.036506
[1] 2.016616
[1] 1
[1] 2.006462
[1] 2.001231
[1] 2.017035
[1] 1
[1] 1
[1] 1
[1] 2.002141
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 2.077105
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 4.000497
[1] 2.068727
[1] 2.060701
[1] 2.031984
[1] 2.009014
[1] 1
[1] 2.005998
[1] 2.005811
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 2.001951
[1] 1
[1] 1
[1] 1
[1] 1
[1] 2.000001
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 2.104315
[1] 2.066741
[1] 2.044846
[1] 3.24474
[1] 1
[1] 2.05809
[1] 2.178834
[1] 2.235389
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 1
[1] 2.999703
[1] 1
[1] 2.999659
[1] 1
[1] 2.0012
[1] 2.000011


100%|██████████| 1000/1000 [00:00<00:00, 1964.73it/s]

Score: BIC, Data: tueb, AUC: 0.6515008163265307


In [14]:
print(resultlist)

[('AIC', 'tueb', 0.6529342857142857), ('BIC', 'tueb', 0.6515008163265307)]


In [ ]:
print(resultlist)

[('AIC', 'cha', 0.6009860888888889), ('AIC', 'multi', 0.9568565777777779), ('AIC', 'net', 0.793262888888889), ('AIC', 'gauss', 0.714019111111111), ('BIC', 'cha', 0.5767421333333334), ('BIC', 'multi', 0.961397288888889), ('BIC', 'net', 0.7118214666666668), ('BIC', 'gauss', 0.6939331555555555)]


In [ ]:
# Get the weighted accuracy
correct_weight = []
wrong_weight = []
for run_idx in list(res_dict.keys()):
    curr_score = res_dict[run_idx]
    if curr_score < 0:
        print(run_idx)
        wrong_weight.append(weight[run_idx])
    if curr_score > 0:
        correct_weight.append(weight[run_idx])

NameError: name 'res_dict' is not defined

In [ ]:
# Find weighted accuracy
correct_total_weight = np.sum(correct_weight)
wrong_total_weight = np.sum(wrong_weight)
accuracy = correct_total_weight / (correct_total_weight + wrong_total_weight)
print(accuracy)